In [1]:
import gymnasium as gym
import numpy as np
from typing import Any, Dict, List, Tuple, Optional
from ray.rllib.env.multi_agent_env import MultiAgentEnv

2023-08-10 12:55:51,530	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
from ray.tune.registry import register_env
from ray.rllib.algorithms import ppo
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.examples.policy.random_policy import RandomPolicy
from rl.envs import OthelloEnv

register_env("othello", lambda _: OthelloEnv({}))

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    agent_id = int(agent_id[-1]) - 1
    return "agent_1" if episode.episode_id % 2 == agent_id else "agent_2"

config = ppo.PPOConfig().environment("othello").framework("torch").rollouts(num_rollout_workers=8)
config = config.multi_agent(policies={"agent_1": PolicySpec(), "agent_2": PolicySpec()}, policy_mapping_fn=policy_mapping_fn, policies_to_train=["agent_1"])
config = config.training(model={"conv_filters": [[32, [3, 3], 1], [64, [3, 3], 1]]})

2023-08-10 12:56:11,426	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [4]:
from ray import air
from ray import tune

results = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
        )
    ),
).fit()

2023-08-10 12:56:16,232	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=21564) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=21564) 2023-08-10 12:56:20,671	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=

In [ ]:
results

ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': '79536_00000'},
    path='c://\\Users\\yoshi\\ray_results\\PPO\\PPO_othello_79536_00000_0_2023-08-10_12-53-32',
    checkpoint=None
  )
]>

In [ ]:
checkpoint = results.get_best_result().checkpoint
print(checkpoint)

None


(PPO pid=23440) 2023-08-10 12:53:41,398	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=24508, ip=127.0.0.1, actor_id=0752922d0a6b44d53ffa963b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000017992C9F100>)
(PPO pid=23440)   File "c:\Users\yoshi\Documents\Repository\rl-mini-app\rl\envs.py", line 134, in step
(PPO pid=23440)     assert (
(PPO pid=23440) AssertionError: Only one agent can take action at a time. {}
(PPO pid=23440) 
(PPO pid=23440) The above exception was the direct cause of the following exception:
(PPO pid=23440) 
(PPO pid=23440) ray::RolloutWorker.__init__() (pid=24508, ip=127.0.0.1, actor_id=0752922d0a6b44d53ffa963b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000017992C9F100>)
(PPO pid=23440)   File "c:\Users\yoshi\miniconda3\envs\rl\lib\site-packages\ray\rllib\utils\pre_checks\env.p

In [ ]:
import random
# multiagent environment for othello
env = OthelloEnv({})
obs, _ = env.reset()
terminated = False
current_player = "agent_1"
while not terminated:
    env.render()
    # action = env.action_space.sample()
    valid_actions = env.get_valid_moves(current_player)
    if len(valid_actions) > 0:
        action = random.choice(valid_actions)
    else:
        action = 64
    action = {current_player: action}
    obs, reward, terminated, truncated, _= env.step(action)
    obs = obs["agent_1" if current_player == "agent_2" else "agent_2"]
    reward = reward[current_player]
    terminated = terminated[current_player]
    truncated = truncated[current_player]
    current_player = env.current_player
    print(terminated, truncated, reward)

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

False False 0.046875
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|O|O|.|.|.
.|.|.|O|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

False False -0.0
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|O|X|.|.|.
.|.|.|O|X|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

False False 0.046875
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|O|.|.
.|.|.|O|O|.|.|.
.|.|.|O|X|.|.|.
.|.|.|O|X|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

False False -0.0
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|O|.|.
.|.|.|O|O|.|.|.
.|.|.|O|X|.|.|.
.|.|.|X|X|.|.|.
.|.|X|.|.|.|.|.
.|.|.|.|.|.|.|.

False False 0.046875
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|O|.|.
.|.|.|O|O|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|X|.|.|.
.|.|X|.|.|.|.|.
.|.|.|.|.|.|.|.

False False -0.0
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|O|.|.
.|.|.|O|O|.|X|.
.|.|.|O|O|X|.|.
.|.|.|X|X|.|.|.
.|.|X|.|.|.|.|.


In [ ]:
env.render()

O|O|O|O|O|O|O|O
X|O|O|O|O|O|O|O
X|O|O|O|O|O|O|O
O|O|X|O|O|X|O|O
X|X|X|X|O|O|X|O
X|X|X|X|X|O|X|O
X|X|X|X|X|X|X|O
O|O|O|O|O|O|X|O

